In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "3"

import sys
sys.path.append('/home/rustam/hyperspecter_segmentation/makitorch')
sys.path.append('/home/rustam/hyperspecter_segmentation/')

In [ ]:
from makitorch import *

In [ ]:
import numpy as np
import comet_ml
import pytorch_lightning as pl
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.loggers import TensorBoardLogger
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms as T
import torchvision.transforms.functional as TF
from torchvision import utils
import cv2
from Losses import FocalLoss
import matplotlib.pyplot as plt

import seaborn as sns
import json

In [ ]:
from sklearn.decomposition import PCA
from makitorch.architectures.U2Net import U2Net

In [ ]:
from hsi_dataset_api import HsiDataset

In [ ]:
from makitorch.dataloaders.HsiDataloader import HsiDataloader
from makitorch.architectures.Unet import Unet, UnetWithFeatureSelection
from makitorch.loss import muti_bce_loss_fusion

In [ ]:
def iou_pytorch(outputs: torch.Tensor, labels: torch.Tensor):
    # You can comment out this line if you are passing tensors of equal shape
    # But if you are passing output from UNet or something it will most probably
    # be with the BATCH x 1 x H x W shape
    outputs = torch.sigmoid(outputs)
    iou_score = np.zeros(outputs.shape[1])
    outputs[outputs > 0.5] = 1
    outputs[outputs <= 0.5] = 0
    num_classes = outputs.shape[1]
    for _cls in range(num_classes):
        output = outputs[::, _cls]
        label = torch.clone(labels[::])
        label[label != (_cls + 1)] = 0
        label[label == (_cls + 1)] = 1
        intersection = torch.logical_and(label, output)
        union = torch.logical_or(label, output)
        iou_score[_cls] = torch.sum(intersection) / (torch.sum(union) + 1e-10)
    return iou_score

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
        if getattr(m, 'bias') is not None:
            m.bias.data.fill_(0.01)

In [ ]:
class WeightConstraint(object):
    def __init__(self):
        pass
    
    def __call__(self,module):
        if hasattr(module,'weight'):
            w=module.weight.data
            w=w.clamp(0, 1)
            module.weight.data=w

In [ ]:
class CustomLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.ce = nn.CrossEntropyLoss()
        
    def forward(self, fs_weight, preds, mask):
        return self.ce(preds, mask) + torch.sum(1 - (torch.abs(fs_weight) / 0.99 - 1) ** 2)

In [ ]:
class NnModel(pl.LightningModule):
    def __init__(self, model, loss, enable_image_logging=False):
        super().__init__()
        self.model = model
        self.loss = loss
        self.enable_image_logging = enable_image_logging
        #self.weight_contraint_function = WeightConstraint()

    def _custom_histogram_adder(self):
        for name,params in self.named_parameters():
            self.logger.experiment.add_histogram(name,params,self.current_epoch)
            
    def forward(self, x):
        out = self.model(x)
        return out
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=4e-3)
        lr_scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.999)
        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}
    
    def training_step(self, train_batch, batch_idx):
        img, mask = train_batch
        preds = self.model(img)
        loss = self.loss(preds, mask)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, val_batch, batch_idx):
        img, mask = val_batch
        preds = self.model(img)
        loss = self.loss(preds, mask)
        metric = iou_pytorch(preds, mask)
        
        if self.enable_image_logging:
            pred = torch.argmax(preds.detach().cpu()[0], dim=0)

            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
            sns.heatmap(pred, ax=ax1, vmin=0, vmax=17)
            sns.heatmap(mask.cpu()[0], ax=ax2, vmin=0, vmax=17)
            fig.savefig('temp_fig.png')
            plt.close(fig)
            
#             trainer.logger.experiment.log_histogram_3d(
#                 self.model.features_selection.weight.detach().cpu().numpy(),
#                 name='band-selection layer',
#                 step=self.global_step
#             )
            if hasattr(trainer.logger.experiment, 'log_image'):
                # For Comet logger
                trainer.logger.experiment.log_image('temp_fig.png', name=f'{batch_idx}', overwrite=False, step=self.global_step)
            else:
                # For tensorboard logger
                img = cv2.imread('temp_fig.png')
                trainer.logger.experiment.add_image(f'{batch_idx}', img, dataformats='HWC')
        
        d = {f'iou_{i}': iou for i, iou in enumerate(metric)}
        d['val_loss'] = loss
        self.log_dict(d, on_step=False, on_epoch=True, prog_bar=True)

In [ ]:
device = 'cuda:0'

In [ ]:
pca_explained_variance = np.load('PcaExplainedVariance_.npy')
pca_mean = np.load('PcaMean.npy')
pca_components = np.load('PcaComponents.npy')

In [ ]:
def pca_transformation(x):
    x_t = np.reshape(x, (x.shape[0], -1)) # (C, H, W) -> (C, H * W)
    x_t = np.swapaxes(x_t, 0, 1) # (C, H * W) -> (H * W, C)
    x_t = x_t - pca_mean
    x_t = np.dot(x_t, pca_components.T) / np.sqrt(pca_explained_variance)
    return np.reshape(x_t, (x.shape[1], x.shape[2], pca_components.shape[0])).astype(np.float32) # (H, W, N)

In [ ]:
def preprocessing(imgs, masks):
    target_size = (256, 256)
    _images = [image.resize(target_size,Image.BILINEAR)
                   for image in imgs]
    _masks = [mask.resize(target_size, Image.BILINEAR) for mask in masks]
    return _images, _masks

In [ ]:
def preprocessing(imgs, masks):
    with open('data_standartization_params.json', 'r') as f:
        data_standartization_params = json.load(f)
    mean = data_standartization_params.get('means')
    std = data_standartization_params.get('stds')
    def standartization(img):
        return np.array((img - mean) / std, dtype=np.float32)
    
    _images = [pca_transformation(image) for image in imgs]
    _images = [standartization(image) for image in _images]
    _masks = [np.expand_dims(cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY).astype(np.int64), 0) for mask in masks]
    return _images, _masks

In [ ]:
def test_augmentation(image, mask):
    image = TF.to_tensor(image)
    image = (image - image.min()) / (image.max() - image.min())
    
    mask = torch.from_numpy(mask)
    
    mask = torch.squeeze(mask, 0)
    return image, mask

In [ ]:
def augmentation(image, mask):
    image = TF.to_tensor(image)
    mask = torch.from_numpy(mask)
    print('mask=',mask.shape, ' image=', image.shape)
    #angle = T.RandomRotation.get_params((-30, 30))
    #image = TF.rotate(image, angle, interpolation=T.InterpolationMode.BILINEAR)
    #mask = TF.rotate(mask, angle, interpolation=T.InterpolationMode.NEAREST)
    
    if np.random.random() > 0.01:
        image = TF.hflip(image)
        mask = TF.hflip(mask)
    print('first flip  mask=',mask.shape, ' image=', image.shape)

    if np.random.random() > 0.01:
        image = TF.vflip(image)
        mask = TF.vflip(mask)
    print('second flip  mask=',mask.shape, ' image=', image.shape)
    
    image = (image - image.min()) / (image.max() - image.min())
    mask = torch.squeeze(mask, 0)
    return image, mask

In [ ]:
# random = np.random.permutation(np.arange(384))
# test_indices = random[310:]
# train_indices = random[:310]

test_indices = np.load('test_indices.npy')
train_indices = np.load('train_indices.npy')
path = '/raid/rustam/hyperspectral_dataset/cropped_hsi_data'

dataset_train = HsiDataloader(path, preprocessing=preprocessing, augmentation=augmentation, indices=train_indices)
dataset_test = HsiDataloader(path, preprocessing=preprocessing, augmentation=test_augmentation, indices=test_indices)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=4)

In [ ]:
val_loader = torch.utils.data.DataLoader(dataset_test, batch_size=1)

In [ ]:
iter_train = iter(train_loader)

In [ ]:
data = next(iter_train)

In [ ]:
data[0].shape

In [ ]:
data[0].cpu().detach().numpy().shape, data[1].cpu().detach().numpy().shape

In [ ]:
sns.heatmap(data[1][0])

In [ ]:
plt.imshow(np.transpose(data[0][0].cpu().detach().numpy(), (1, 2, 0))[..., 0])

In [ ]:
sns.heatmap(np.transpose(data[0][0].cpu().detach().numpy(), (1, 2, 0))[..., 0])

In [ ]:
data = next(iter(val_loader))

In [ ]:
sns.heatmap(np.transpose(data[0][0].cpu().detach().numpy(), (1, 2, 0))[..., 0])

In [ ]:
data[1].shape

In [ ]:
net = Unet(in_channels=17, out_channels=17, init_features=128, pretrained=False)
# net = net.apply(init_weights)
    # net = UnetWithFeatureSelection(in_channels=236, out_channels=17, init_features=128, pretrained=False)

# net = U2Net(17, 17)

In [ ]:
class MySuperNet(nn.Module):
    
    def __init__(self, end_net: Unet, in_f=17):
        super().__init__()
        self.end_net = end_net
        self.bn_start = nn.BatchNorm2d(in_f)
    
    def __call__(self, x):
        x = self.bn_start(x)
        x = self.end_net(x)
        return x

In [ ]:
net = MySuperNet(net, 17)

In [ ]:
logger = CometLogger(
    api_key="your-key",
    workspace="your-workspace",  # Optional
    project_name="your-project-name",  # Optional
    experiment_name="Norm image min/max // maki-style FC (gamma=4, normed by positive balance=2)"
)

#logger = TensorBoardLogger(
#    'logs/'
#)


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLossCustom(nn.Module):
    """
    copy from: https://github.com/Hsuxu/Loss_ToolBox-PyTorch/blob/master/FocalLoss/FocalLoss.py
    This is a implementation of Focal Loss with smooth label cross entropy supported which is proposed in
    'Focal Loss for Dense Object Detection. (https://arxiv.org/abs/1708.02002)'
        Focal_Loss= -1*alpha*(1-pt)*log(pt)
    :param num_class:
    :param alpha: (tensor) 3D or 4D the scalar factor for this criterion
    :param gamma: (float,double) gamma > 0 reduces the relative loss for well-classified examples (p>0.5) putting more
                    focus on hard misclassified example
    :param smooth: (float,double) smooth value when cross entropy
    :param balance_index: (int) balance class index, should be specific when alpha is float
    :param size_average: (bool, optional) By default, the losses are averaged over each loss element in the batch.
    """

    def __init__(self, alpha=None, gamma=2, balance_index=2, smooth=1e-5, size_average=False):
        super(FocalLossCustom, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.balance_index = balance_index
        self.smooth = smooth
        self.size_average = size_average
        self.cel = nn.CrossEntropyLoss(reduction='none')
        self.softmax = nn.Softmax(dim=-1)

        if self.smooth is not None:
            if self.smooth < 0 or self.smooth > 1.0:
                raise ValueError('smooth value should be in [0,1]')

    def forward(self, logit, target):
        num_class = logit.shape[1]

        if logit.dim() > 2:
            # N,C,d1,d2 -> N,C,m (m=d1*d2*...)
            logit = logit.view(logit.size(0), logit.size(1), -1)
            logit = logit.permute(0, 2, 1).contiguous()
            logit = logit.view(-1, logit.size(-1))
        target = torch.squeeze(target, 1)
        target = target.view(-1, 1)
        
        ce_loss = self.cel(logit, target.view(-1))
        train_conf = self.softmax(logit)
        
        idx = target.cpu().long()
        one_hot_labels  = torch.FloatTensor(target.size(0), num_class).zero_()
        one_hot_labels  = one_hot_labels.scatter_(1, idx, 1)
        if one_hot_labels.device != logit.device:
            one_hot_labels = one_hot_labels.to(logit.device)
        
        filtered_conf = train_conf * one_hot_labels
        sparce_conf, _ = torch.max(filtered_conf, dim=-1)
        loss = torch.pow((torch.ones_like(sparce_conf) - sparce_conf), self.gamma) * ce_loss
        """
        
        # print(logit.shape, target.shape)
        # 
        alpha = self.alpha

        if alpha is None:
            alpha = torch.ones(num_class, 1)
        elif isinstance(alpha, (list, np.ndarray)):
            assert len(alpha) == num_class
            alpha = torch.FloatTensor(alpha).view(num_class, 1)
            alpha = alpha / alpha.sum()
        elif isinstance(alpha, float):
            alpha = torch.ones(num_class, 1)
            alpha = alpha * (1 - self.alpha)
            alpha[self.balance_index] = self.alpha

        else:
            raise TypeError('Not support alpha type')
        
        if alpha.device != logit.device:
            alpha = alpha.to(logit.device)

        idx = target.cpu().long()

        one_hot_key = torch.FloatTensor(target.size(0), num_class).zero_()
        one_hot_key = one_hot_key.scatter_(1, idx, 1)
        if one_hot_key.device != logit.device:
            one_hot_key = one_hot_key.to(logit.device)

        if self.smooth:
            one_hot_key = torch.clamp(
                one_hot_key, self.smooth/(num_class-1), 1.0 - self.smooth)
        pt = (one_hot_key * logit).sum(1) + self.smooth
        logpt = pt.log()
        
        gamma = self.gamma

        alpha = alpha[idx]
        alpha = torch.squeeze(alpha)
        loss = -1 * alpha * torch.pow((1 - pt), gamma) * logpt
        """
        if self.size_average:
            loss = loss.mean()
        if not self.size_average:
            # Norm by positive
            num_positive = torch.sum(target != self.balance_index)
            loss = loss.sum() / (num_positive + 1e-10)
        else:
            loss = loss.sum()
        return loss

In [ ]:
# model = NnModel(net, muti_bce_loss_fusion, enable_image_logging=True)
model = NnModel(net, FocalLossCustom(), enable_image_logging=True)

trainer = pl.Trainer(
    gpus=1, 
    max_epochs=14,
    check_val_every_n_epoch=2,
    logger=logger
)
# trainer = pl.Trainer(
#     gpus=1, 
#     max_epochs=2000,
#     check_val_every_n_epoch=2000)
trainer.fit(model, train_loader, val_loader)

In [ ]:
msg = """

"""

In [ ]:
logger.experiment.log_html(msg)

In [ ]:
logger.experiment.end()

In [ ]:
dir(logger)

In [ ]:
plt.plot(net.features_selection.weight.detach().cpu().numpy()[::, 0, 0, 0])

In [ ]:
import itertools

In [ ]:
mean = np.zeros((17, ))
std = np.zeros((17, ))
count = 0
for img, mask in train_loader:
    img = img.detach().cpu().numpy()
    mean += img.mean(axis=(2, 3))[0]
    std += img.std(axis=(2, 3))[0]
    count += 1
mean /= count
std /= count

In [ ]:
# with open('data_standartization_params.json', 'w') as f:
#     json.dump(
#         {
#             "means": mean.tolist(),
#             "stds": std.tolist()
#         }, f
#     )

In [ ]:
x, y = next(itertools.islice(train_loader.__iter__(), 0, None))

In [ ]:
x.shape

In [ ]:
preds = net.forward(x)

In [ ]:
preds = preds.detach().cpu().numpy()[0]

In [ ]:
fig, axis = plt.subplots(17, 1, figsize=(6, 30))
for i, ax in enumerate(axis):
    sns.heatmap(preds[i], ax=ax, vmin=0, vmax=1)

In [ ]:
# PyTroch version

SMOOTH = 1e-6

def iou_pytorch(outputs: torch.Tensor, labels: torch.Tensor):
    # You can comment out this line if you are passing tensors of equal shape
    # But if you are passing output from UNet or something it will most probably
    # be with the BATCH x 1 x H x W shape
    outputs = outputs.squeeze(1)  # BATCH x 1 x H x W => BATCH x H x W
    
    intersection = (outputs & labels).float().sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
    union = (outputs | labels).float().sum((1, 2))         # Will be zzero if both are 0
    
    iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0
    
    thresholded = torch.clamp(20 * (iou - 0.5), 0, 10).ceil() / 10  # This is equal to comparing with thresolds
    
    return thresholded  # Or thresholded.mean() if you are interested in average across the batch
    
    
# Numpy version
# Well, it's the same function, so I'm going to omit the comments

def iou_numpy(outputs: np.array, labels: np.array):
    outputs = outputs.squeeze(1)
    
    intersection = (outputs & labels).sum((1, 2))
    union = (outputs | labels).sum((1, 2))
    
    iou = (intersection + SMOOTH) / (union + SMOOTH)
    
    thresholded = np.ceil(np.clip(20 * (iou - 0.5), 0, 10)) / 10
    
    return thresholded  # Or thresholded.mean()

In [ ]:
iter_val = iter(val_loader)

In [ ]:
in_data_x, val_data = next(iter_val)

In [ ]:
preds = net(in_data_x)

In [ ]:
preds.shape, val_data.shape

In [ ]:
preds_np, val_data_np = preds.detach().numpy(), val_data.detach().numpy()
preds_np.shape, val_data_np.shape

In [ ]:
preds_np = np.argmax(preds_np, axis=1)
preds_np.shape

In [ ]:
iou_numpy(np.expand_dims(preds_np, axis=0), val_data_np)

In [ ]:
preds_np

In [ ]:
val_data_np

In [ ]:
sns.heatmap(preds_np[0])

In [ ]:
sns.heatmap(val_data_np[0])

In [ ]:
plt.imshow(in_data_x[0, 0])